run luenberger estimator

In [ ]:
import sys, os
import importlib
import numpy as np

sys.path.append('../')

import m_params
importlib.reload(m_params)
from m_params import params

import m_target
importlib.reload(m_target)
from m_target import target             

import m_radar
importlib.reload(m_radar)
from m_radar import c_radar             

import m_filtertype
importlib.reload(m_filtertype)

# initial conditions 
z0 = 100000 * params.ft2m
v0 = -6000 * params.ft2m
beta = 500 * params.lbft2kgm

rdr = c_radar(np.array([z0 + 25 * params.ft2m, v0 - 150 * params.ft2m, beta + 300 * params.lbft2kgm])
              , m_filtertype.filtertype.luenberger)


print('is observable:')
print(rdr.filter.isobservable())
print('desired eigen values:')
print(rdr.filter.eig())
print('desired eigen values:')
print(np.array([-4, -4, -4]))

rdr.filter.setest(np.array([-4, -4, -4]))




In [23]:
import m_params         # this one is for the reload 
import m_models         # this one is for the reload 

from m_params import params # for the direct calling  

from scipy.integrate import solve_ivp, odeint

import importlib
importlib.reload(m_params)
importlib.reload(m_models)

# initial conditions 
z0 = 100000 * params.ft2m
v0 = -6000 * params.ft2m
a0 = 0
beta = 500 * params.lbft2kgm

# sim parameters 
dt = 1e-3
tf = 30 
t = np.arange(0, tf, dt)

  
''' simulate nonlinear system '''
y_nln = odeint(m_models.nonlinear, [z0, v0, beta], t) 
y_nlnsimp = odeint(m_models.nonlinear_simp, [z0, v0, beta], t) 
y_lin = odeint(m_models.linear, [z0, v0, beta], t, args = (beta, )) 


In [ ]:
plt.plot(t, y_nlnsimp[:, 0], label = 'simplified nonlinear')
plt.plot(t, y_lin[:, 0], label = 'linear')
plt.legend()
plt.title('Altitude')
plt.show()

plt.plot(t, y_nlnsimp[:, 1], label = 'simplified nonlinear')
plt.plot(t, y_lin[:, 1], label = 'linear')
plt.legend()
plt.title('Velocity')
plt.show()

plt.plot(t, y_nlnsimp[:, 2], label = 'simplified nonlinear')
plt.plot(t, y_lin[:, 2], label = 'linear')
plt.legend()
plt.title(r'$\beta$')
plt.show()


In [ ]:
plt.plot(t, y_nln[:, 0], label = 'nonlinear')
plt.plot(t, y_lin[:, 0], label = 'linear')
plt.legend()
plt.title('Altitude')
plt.show()

plt.plot(t, y_nln[:, 1], label = 'nonlinear')
plt.plot(t, y_lin[:, 1], label = 'linear')
plt.legend()
plt.title('Velocity')
plt.show()

plt.plot(t, y_nln[:, 2], label = 'nonlinear')
plt.plot(t, y_lin[:, 2], label = 'linear')
plt.legend()
plt.title(r'$\beta$')
plt.show()


In [ ]:

from scipy.integrate import solve_ivp, odeint

import importlib
importlib.reload(m_params)
importlib.reload(m_target)
importlib.reload(m_radar)


# initial conditions 
z0 = 100000 * params.ft2m
v0 = -6000 * params.ft2m
a0 = 0
beta = 500 * params.lbft2kgm

# sim parameters 
dt = 1e-3
tf = 30 

# objects definition 
tgt = target([z0, v0, a0], beta)
rdr = radar([z0 + 25 * params.ft2m, v0 - 150 * params.ft2m, beta + 300 * params.lbft2kgm])

# main loop 
for t in np.arange(0, tf, dt):
    
    ''' simulate target motion '''
    y = odeint(target.deriv, [tgt.z, tgt.vz], t + [0, dt], args = (tgt, )) 
    # if y[-1, 0] <= 0:
    #     break
    tgt.z, tgt.vz = y[-1, :]
    tgt.data = np.concatenate((tgt.data, np.array([[tgt.z, tgt.vz, 0, 0]])), axis = 0)
    
    ''' take radar measure '''
    tgt_dist = tgt.z + rdr.vk * np.random.randn(1) #  0.86217 # 
    tgt_measure = rdr.measure(t, tgt_dist)

# zmask = tgt.data[:, 0] < 100000


dy = target.deriv(tgt.data, 0, tgt) 
tgt.data[:, 2] = np.asarray(dy).T[:, 0]
tgt.data[:, 3] = np.ones(len(tgt.data)) * beta